<a href="https://colab.research.google.com/github/rushikeshnaik779/PracticeForNLP/blob/main/TheNewsCategoryClassification/Predict_The_News_Category_Hackathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd


In [2]:
data = pd.read_excel('./Data_Train.xlsx')
data.head()

,STORY,SECTION
0,But the most painful was the huge reversal in ...,3
1,How formidable is the opposition alliance amon...,0
2,Most Asian currencies were trading lower today...,3
3,"If you want to answer any question, click on ‘...",1
4,"In global markets, gold prices edged up today ...",3


In [3]:
test = pd.read_excel('./Data_Test.xlsx')
test.head()

,STORY
0,2019 will see gadgets like gaming smartphones ...
1,It has also unleashed a wave of changes in the...
2,It can be confusing to pick the right smartpho...
3,The mobile application is integrated with a da...
4,We have rounded up some of the gadgets that sh...


In [4]:
data.shape

(7628, 2)

In [5]:
data.SECTION.value_counts()

1    2772
2    1924
0    1686
3    1246
Name: SECTION, dtype: int64

In [6]:
# importing libraries 

import tensorflow as tf 

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


import tensorflow.keras.layers as L 
from tensorflow.keras.losses import SparseCategoricalCrossentropy 
from tensorflow.keras.optimizers import Adam


from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import classification_report

import matplotlib.pyplot as plt 
import seaborn as sns

import numpy as np
import pandas as pd

import nltk 
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import TweetTokenizer, word_tokenize
from nltk.corpus import stopwords

import re

In [7]:
# Cleaning 
nltk.download('stopwords')
stop_words = stopwords.words('english')

def data_cleaner(tweet):
    # remove urls
    tweet = re.sub(r'http\S+', '', tweet)

    # remove html tags 
    tweet = re.sub(r'<.*?>', '', tweet)

    # remove digits
    tweet = re.sub(r'\d+', '', tweet)

    #remove hashtags 
    tweet = re.sub(r'#\w+', '', tweet)

    # remove mentions 
    tweet = re.sub(r'@\w+', '', tweet)

    # removing stop words

    tweet = tweet.split()
    tweet = " ".join([word for word in tweet if not word in stop_words])

    return tweet

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [17]:
X = data['STORY']
y = data['SECTION']

In [18]:
X_cleaned = X.apply(data_cleaner)
X_cleaned.head()

0    But painful huge reversal fee income, unheard ...
1    How formidable opposition alliance among Congr...
2    Most Asian currencies trading lower today. Sou...
3    If want answer question, click ‘Answer’. After...
4    In global markets, gold prices edged today dis...
Name: STORY, dtype: object

In [19]:
tokenizer = Tokenizer()

tokenizer.fit_on_texts(X_cleaned)

X = tokenizer.texts_to_sequences(X_cleaned)
vocab_size = len(tokenizer.word_index)+1

print("Vocabulary size {}".format(vocab_size))
print("\nExample:\n")
print("Sentence:\n{}".format(X_cleaned[10]))
print("\nAfter tokenizing :\n{}".format(X[10]))

X = pad_sequences(X, padding='post')
print("\nAfter padding: \n{}".format(X[10]))

Vocabulary size 35925

Example:

Sentence:
“One would think development testing process would’ve uncovered many flaws, yet proceeded put market anyway," said Bryan Ma, vice president devices research consultancy IDC. “Clearly can’t afford another embarrassing Note -like incident, lest build reputation releasing unreliable products." The Note episode triggered global recall, cost company billions dollars marred reputation battled Apple Inc. premium devices. Pulling Fold lets Korean giant address potential issues races put flexible gadget ahead Chinese rival Huawei Technologies Co. Xiaomi Corp. A Samsung spokeswoman declined comment story. Shares company, hasn’t set another date commercial launch, little changed Seoul Wednesday. Samsung bounced back since Note — remains world’s largest producer smartphones memory chips. But counting folding devices extend lead mobile kick-start stagnating global market. Unveiled along th-anniversary version flagship Galaxy S phone, Fold underscored Samsu

In [41]:

tf.keras.backend.clear_session()

# hyper parameters 
EPOCHS = 4 
BATCH_SIZE = 64 
embedding_dim = 32
units = 150

model = tf.keras.Sequential([
        L.Embedding(vocab_size, embedding_dim, input_length=X.shape[1]),
        L.Bidirectional(L.LSTM(units, return_sequences=True)),
        L.Bidirectional(L.LSTM(units, return_sequences=True)),
       # L.Bidirectional(L.LSTM(units, return_sequences=True)),




        L.GlobalMaxPooling1D(),
        L.Dropout(0.6),
        L.Dense(128, activation='relu'),
        L.Dense(4)


])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 664, 32)           1149600   
_________________________________________________________________
bidirectional (Bidirectional (None, 664, 300)          219600    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 664, 300)          541200    
_________________________________________________________________
global_max_pooling1d (Global (None, 300)               0         
_________________________________________________________________
dropout (Dropout)            (None, 300)               0         
_________________________________________________________________
dense (Dense)                (None, 128)               38528     
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 5

In [42]:
model.compile(loss=SparseCategoricalCrossentropy(from_logits=True),
              optimizer='adam', 
              metrics=['accuracy'])

In [43]:

training_history = model.fit(X, y, epochs=EPOCHS, validation_split=0.12,
                             batch_size=BATCH_SIZE)

Epoch 1/4
38/38 [==============================] - 328s 8s/step - loss: 1.2652 - accuracy: 0.4346 - val_loss: 1.0008 - val_accuracy: 0.5273
Epoch 2/4
38/38 [==============================] - 320s 8s/step - loss: 0.7330 - accuracy: 0.6764 - val_loss: 0.5320 - val_accuracy: 0.7909
Epoch 3/4
38/38 [==============================] - 318s 8s/step - loss: 0.4162 - accuracy: 0.8426 - val_loss: 0.6664 - val_accuracy: 0.8030
Epoch 4/4
38/38 [==============================] - 320s 8s/step - loss: 0.2608 - accuracy: 0.8980 - val_loss: 0.6707 - val_accuracy: 0.8364


In [44]:
X = test['STORY']
X_cleaned = X.apply(data_cleaner)
X_cleaned.head()


X = tokenizer.texts_to_sequences(X_cleaned)
vocab_size = len(tokenizer.word_index)+1

print("Vocabulary size {}".format(vocab_size))
print("\nExample:\n")
print("Sentence:\n{}".format(X_cleaned[10]))
print("\nAfter tokenizing :\n{}".format(X[10]))

X = pad_sequences(X, padding='post')
print("\nAfter padding: \n{}".format(X[10]))

Vocabulary size 35925

Example:

Sentence:
The NCP leadership tremendous pressure consider fielding third-generation Pawars key constituencies western Maharashtra. Parth Pawar open keenness contest Maval Lok Sabha constituency near Pune. Rohit Pawar, son Sharad Pawar’s brother Rajendra Pawar, also keen entering electoral arena set eyes Karjat-Jamkhed assembly constituency Ahmednagar constituency. The pressure increased Sharad Pawar said would contest Madha Sule would seek re-election Baramati. Last month, Sharad Pawar sought end speculation saying third-generation Pawar would contest Lok Sabha polls. However, NCP leaders legislators say Parth Pawar Rohit Pawar ready wait. Before Pawar intervened, Maval constituency saw posters Parth Pawar coming several places. Known ambitious two third-generation ticket aspirants, Parth started touring constituency six months back, one NCP legislator said. “He keen get active politics. He dedicated band young workers confident win Maval seat time too,

In [45]:
y = model.predict_classes(X)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [46]:
y = list(y)
y = pd.DataFrame({'SECTION':y})
y.to_excel('Sample_submission.xlsx', index=False)